In [1]:
import dash_leaflet as dl
from dash import html, Dash, dash_table, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter()


# class find method must support return of list object and accept projection json input
# sending the find method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.find({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

app.layout = html.Div([
    html.Center([
            html.A(href="www.snhu.edu", children=[
                html.Img(
                    src="/assets/grazioso_salvare_logo.png", 
                    style={"height": 80, "width": 80, "backgroundColor": "white", "borderRadius": "50%", "marginRight": '10px'}
                )]
            ),
            html.B(html.H1('Grazioso Salvare Dashboard', style={'color': 'white'})),
            html.H5("Developed by Aidan Farhi", style={'color': 'white', 'marginLeft': '10px', 'fontStyle': 'italic'})
        ],
        style={"display": "flex", "flexDirection": "row", "alignItems": "center", "justifyContent": "center"}
    ),
    html.Hr(),
    dcc.Dropdown(
        id="filter-dropdown",
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
        ],
        value="All",
        placeholder="Select a category to filter",
        style={'width': '30vw', 'marginBottom': '10px'}
    ),
    html.Div([
        html.Div(
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                page_size=5,
                row_selectable='single',
                selected_rows=[0],
                virtualization=True,
                page_action='none',
                fixed_rows={'headers': True},
                style_table={
                    'overflowY': 'scroll',
                    'overflowX': 'scroll',
                    'height': '30vh'
                },
                style_header={
                    'backgroundColor': "#FFFFFF",
                    'color': "#181818",
                    'fontWeight': 'bold',
                    'border': '1px solid #181818'
                },
                style_cell={
                    'backgroundColor': "white",
                    'color': '#181818',
                    'border': '1px solid #181818',
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'width': '200px',
                    'overflow': 'hidden',
                    'textOverflow': 'ellipsis',
                    'fontSize': '11px'
                },
            ), style={'width': '99vw'}
        ),
        html.Div([
            html.Div(id='pie-id', style={'width': '50vw'}),
            html.Div(id='map-id', style={'width': '50vw'})
        ], 
        style={'display': 'flex', 'flexDirection': 'row', 'marginTop': '3px'}
        )
    ]),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if not selected_rows:
        return []
    return [{
        'if': {'row_index': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_rows]


# This callback will respond to options selected from the dropdown
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-dropdown', 'value')]
)
def filter_table(filter_value):
    if filter_value == 'All':
        filter_df = df
    elif filter_value == 'Water Rescue':
        query_result = shelter.find(
            {
                'animal_type': 'Dog',
                'breed': {
                    '$in': [
                        'Labrador Retriever Mix',
                        'Chesapeake Bay Retriever',
                        'Newfoundland'
                    ]
                },
                'sex_upon_outcome': 'Intact Female',
                'age_upon_outcome_in_weeks': {
                    '$gte': 26
                },
                'age_upon_outcome_in_weeks': {
                    '$lte': 156
                }
            }
        )
        filter_df = pd.DataFrame.from_records(query_result).drop(columns=['_id'])
    elif filter_value == 'Mountain or Wilderness Rescue':
        query_result = shelter.find(
            {
                'animal_type': 'Dog',
                'breed': {
                    '$in': [
                        'German Shepherd', 
                        'Alaskan Malamute', 
                        'Old English Sheepdog', 
                        'Siberian Husky',
                        'Rottweiler'
                    ]
                },
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {
                    '$gte': 26
                },
                'age_upon_outcome_in_weeks': {
                    '$lte': 156
                }
            }
        )
        filter_df = pd.DataFrame.from_records(query_result).drop(columns=['_id'])
    elif filter_value == 'Disaster Rescue or Individual Tracking':
        query_result = shelter.find(
            {
                'animal_type': 'Dog',
                'breed': {
                    '$in': [
                        'Doberman Pinscher', 
                        'German Shepherd', 
                        'Golden Retriever',
                        'Bloodhound', 
                        'Rottweiler'
                    ]
                },
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {
                    '$gte': 20
                },
                'age_upon_outcome_in_weeks': {
                    '$lte': 300
                }
            }
        )
        filter_df = pd.DataFrame.from_records(query_result).drop(columns=['_id'])
    else:
        filter_df = df
    return filter_df.to_dict('records')


# This callback will populate the pie chart
@app.callback(
    Output('pie-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_pie(viewData):
    dff = pd.DataFrame.from_dict(viewData or [])
    if dff.empty or "breed" not in dff.columns:
        return html.P("No data to display pie chart.")
    pie_fig = px.pie(
        dff,
        names='breed',
        title='Distribution by Breed'
    )
    pie_fig.update_traces(textposition='inside', textinfo='percent+label')

    return dcc.Graph(figure=pie_fig, style={'height': '40vh'})


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData or [])
    if dff.empty or dff.shape[1] < 15:
        return [html.P("No data available or coordinates missing.")]
    row = index[0] if index else 0
    return [
        dl.Map(
            style={'height': '40vh'},
            center=[30.75,-97.48], 
            zoom=9,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row,13],dff.iloc[row,14]], 
                    children=[
                        dl.Tooltip(dff.iloc[row,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row,9])
                        ])
                    ]
                )
            ]
        )
    ]
    
app.run(debug=True, port=9889, jupyter_height=900)